In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nb_001b import *
from PIL import Image
import PIL, matplotlib.pyplot as plt
from torch.utils.data import Dataset
from operator import itemgetter, attrgetter

# Temp storage

In [ ]:
def is_listy(x): return isinstance(x, (list,tuple))

# Carvana

In [ ]:
PATH = Path('data/carvana')
PATH_PNG = PATH/'train_masks_png'
PATH_X = PATH/'train-128'
PATH_Y = PATH/'train_masks-128'

## Convert and resize data

In [ ]:
PATH_PNG.mkdir(exist_ok=True)
PATH_X.mkdir(exist_ok=True)
PATH_Y.mkdir(exist_ok=True)

In [ ]:
def convert_img(fn): Image.open(fn).save(PATH_PNG/f'{fn.name[:-4]}.png')

In [ ]:
files = list((PATH/'train_masks').iterdir())
with ThreadPoolExecutor(8) as e: e.map(convert_img, files)

In [ ]:
def resize_img(fn, dirname):
    Image.open(fn).resize((128,128)).save((fn.parent.parent)/dirname/fn.name)

In [ ]:
files = list(PATH_PNG).iterdir())
with ThreadPoolExecutor(8) as e: e.map(partial(resize_img, dirname='train_masks-128'), files)

In [ ]:
files = list((PATH/'train').iterdir())
with ThreadPoolExecutor(8) as e: e.map(partial(resize_img, dirname='train-128'), files)

## Basic transforms

In [ ]:
img_f = next(PATH_X.iterdir())
img_x = open_image(img_f)
show_image(img_x)

In [ ]:
def get_y_fn(x_fn): return f'{x_fn[:-4]}_mask.png'

In [ ]:
img_y_f = PATH_Y/get_y_fn(img_f.name)
img_y = open_image(img_y_f)
show_image(img_y)

In [ ]:
def x(): return open_image(img_f)
def y(): return open_image(img_y_f)

In [ ]:
tfms = [flip_lr_tfm(p=0.5),
        rotate_tfm(degrees=(-10,10.), p=0.25),
        zoom_tfm(scale=(0.8,1.2), p=0.25),
        contrast_tfm(scale=(0.8,1.2)),
        brightness_tfm(change=(0.4,0.6))
]

In [ ]:
_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes: show_image(apply_pipeline(x(), tfms), ax)

In [ ]:
_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes: show_image(apply_pipeline(y(), tfms), ax)

# Dependent var transforms

## Rotation

In [ ]:
def xy(): return x(),y()

In [ ]:
resolve_args(brightness, change=(0.4,0.6))

In [ ]:
def rotate_rand(x, y=None, smooth_y=True):
    args = resolve_args(rotate, degrees=(-45,45.))
    m = rotate(**args)
    x = do_affine(x, m)
    if y is None: return x
    
    y = do_affine(y, m)
    if not smooth_y: torch.round_(y)
    return x, y

In [ ]:
imgx,imgy = rotate_rand(*xy(), smooth_y=False)
assert(torch.any((imgy>0.) & (imgy<1.)) == 0)

In [ ]:
_,axes = plt.subplots(2,4, figsize=(12,6))
for i in range(4):
    imgx,imgy = rotate_rand(*xy(), smooth_y=False)
    show_image(imgx, axes[0][i])
    show_image(imgy, axes[1][i])

In [ ]:
_,axes = plt.subplots(2,4, figsize=(12,6))
for i in range(4):
    imgx,imgy = rotate_rand(x(),x())
    show_image(imgx, axes[0][i])
    show_image(imgy, axes[1][i])

In [ ]:
_,axes = plt.subplots(1,4, figsize=(12,6))
for ax in axes: show_image(rotate_rand(x()), ax)

## Affine transforms

In [ ]:
def do_affine(img_x, img_y=None, m=None, funcs=None, smooth_y=True):
    if m is None: m=eye_new(img_x, 3)
    c = affine_grid(img_x,  img_x.new_tensor(m))
    c = compose(funcs)(c)
    img_x = grid_sample(img_x, c, padding='zeros')
    if img_y is None: return img_x

    img_y = grid_sample(img_y, c, padding='zeros')
    if not smooth_y: torch.round_(img_y)
    return img_x, img_y

In [ ]:
def apply_pixel_tfm(func): 
    def _inner(x,y=None):
        logit_(x)
        if y is None: return func(x).sigmoid()
        logit_(y)
        x,y = func(x,y)
        return x.sigmoid(),y.sigmoid()
    
    return _inner

In [ ]:
def apply_pipeline(tfms, x, y=None, smooth_y=True):
    tfms = listify(tfms)
    if len(tfms)==0: return x
    grouped_tfms = dict_groupby(tfms, lambda o: o.__annotations__['return'])
    pixel_tfms,coord_tfms,affine_tfms = map(grouped_tfms.get, TfmType)
    x = apply_pixel_tfm(compose(pixel_tfms))(x,y)
    if isinstance(x,tuple): x,y = x
    matrices = [f() for f in listify(affine_tfms)]
    return do_affine(x, y, affines_mat(x, matrices), funcs=coord_tfms, smooth_y=smooth_y)

In [ ]:
tfms = [rotate_tfm(degrees=(-45,45.)), brightness_tfm(change=(0.3,0.7))]

In [ ]:
imgx,imgy = apply_pipeline(tfms, *xy(), smooth_y=False)
assert(torch.any((imgy>0.) & (imgy<1.)) == 0)

In [ ]:
_,axes = plt.subplots(2,4, figsize=(12,6))
for i in range(4):
    imgx,imgy = apply_pipeline(tfms, *xy(), smooth_y=False)
    show_image(imgx, axes[0][i])
    show_image(imgy, axes[1][i])

In [ ]:
_,axes = plt.subplots(2,4, figsize=(12,6))
for i in range(4):
    imgx,imgy = apply_pipeline(tfms, x(),x())
    show_image(imgx, axes[0][i])
    show_image(imgy, axes[1][i])

In [ ]:
_,axes = plt.subplots(1,4, figsize=(12,6))
for ax in axes: show_image(apply_pipeline(tfms, x()), ax)

In [ ]:
tfms2 = [jitter_tfm(magnitude=(-0.1,0.1))]

_,axes = plt.subplots(1,4, figsize=(12,6))
for ax in axes: show_image(apply_pipeline(tfms2, x()), ax)

# Extending the training loop

## Annealing

In [ ]:
import functools

def annealing_no(start, end, pct): return start
def annealing_linear(start, end, pct): return start + pct * (end-start)
def annealing_exp(start, end, pct): return start * ((end/start) ** pct)
def annealing_cos(start, end, pct):
    cos_out = np.cos(np.pi * pct) + 1
    return end + (start-end)/2 * cos_out
    
def do_annealing_poly(start, end, pct, degree): return end + (start-end) * (1-pct)**degree
def annealing_poly(degree): return functools.partial(do_annealing_poly, degree=degree)

In [ ]:
annealings = "NO LINEAR COS EXP POLY".split()

In [ ]:
a = np.arange(0, 100)
p = np.linspace(0.01,1,100)

In [ ]:
import matplotlib.pyplot as plt

fns = [annealing_no, annealing_linear, annealing_cos, annealing_exp, annealing_poly(0.8)]
for fn, t in zip(fns, annealings):
    plt.plot(a, [fn(2, 1e-2, o) for o in p], label=t)
plt.legend();

## Hyperparam scheduler

In [ ]:
def set_val_in_group(val, group, key):
    if key == 'momentum' and 'betas' in group:  group['betas'] = (val, group['betas'][1])
    elif key == 'beta' and 'betas' in group:    group['betas'] = (group['betas'][0], val)
    elif key == 'beta' and 'alpha' in group:    group['alpha'] = val
    else:                                       group[key] = val

In [ ]:
class HyperParamScheduler():
    def __init__(self, opt, hyper_param, n_iter, annealing, start, end=None, extra=None):
        self.opt,self.hyper_param,self.n_iter,self.annealing = opt,hyper_param,n_iter,annealing
        self.start,self.end,self.extra = start,end,extra
        # TODO: what's this?
        #if None, default end is 0 (only useful for cosine annealing, so we can decide to remove this)
        #at the same time, defaut arg can't be 0 since annealing_exp wouldn't like it. So this is
        #is a messy way around.
        if self.end is None and not (self.annealing in [annealing_no, annealing_exp]):
            self.end = 0
        self.reset()
    
    def reset(self):
        self.n = 0
        self.vals = []
        self.set_val(self.start)
        
    def step(self):
        self.n += 1
        # TODO: what's this?
        #This is the class telling anyone listening it's finished: useful for when you want to go the next phase.
        if self.n >= self.n_iter: return True
        lr = self.annealing(self.start, self.end, self.n/(self.n_iter-1))
        self.set_val(lr)
        return False
    
    def set_val(self, val):
        if not is_iter(val): val = [val]
        if len(val) == 1: val = val * len(self.opt.param_groups)
        self.vals.append(val[-1])
        for v, g in zip(val,self.opt.param_groups): set_val_in_group(v, g, self.hyper_param)

In [ ]:
model, opt = get_model()

## lr_find

In [ ]:
import collections

In [ ]:
MODEL_PATH = PATH/'models'
MODEL_PATH.mkdir(exist_ok=True)

TEMP_MODEL_NAME = 'tmp.pt'

In [ ]:
def save_model(model, fname): torch.save(model.state_dict(), fname)
def load_model(model, fname): model.load_state_dict(torch.load(fname))

In [ ]:
def lr_find(model, loss_fn, opt_fn, train_dl, n_iter=200, start_lr=1e-5, end_lr=10):
    opt = opt_fn(model.parameters(), lr=start_lr)
    
    save_model(model, MODEL_PATH/'tmp.pt')
    model.train()
    lrs, losses= [], []
    msa_loss, beta = 0, 0.98
    min_loss = 0
    lr_sched = HyperParamScheduler(opt, 'lr', n_iter, annealing_exp, start_lr, end_lr)
    finished = False
    while not finished:
        for xb,yb in train_dl: 
            #pdb.set_trace()
            raw_loss,_ = loss_batch(model, xb, yb, loss_fn, opt)
            # smoothes the loss with a mean average
            msa_loss = beta * msa_loss + (1-beta) * raw_loss
            losses.append(msa_loss / (1 - beta ** (len(losses)+1)))
            finished = lr_sched.step()
            if len(losses) == 1 or msa_loss < min_loss: min_loss = msa_loss
            if finished or msa_loss > 4 * min_loss: break

    load_model(model, MDL_PATH/'tmp.pt')
    return lr_sched.vals, losses

In [ ]:
def plot_lr(lrs, losses, skip_start=10, skip_end=5):
    fig, ax = plt.subplots(1)
    lrs = lrs[skip_start:-skip_end] if skip_end !=0 else lrs[skip_start:]
    losses = losses[skip_start:-skip_end] if skip_end !=0 else losses[skip_start:]
    ax.plot(lrs, losses)
    ax.set_xscale('log')

In [ ]:
model, opt = get_model()

In [ ]:
lrs, losses = lr_find(model, loss_fn, optim.SGD, data.train_dl, end_lr=100)
plot_lr(lrs, losses)

## Refactors inside to use the fit function

In [ ]:
class Callback():
    def on_train_begin(self): pass
    def on_train_end(self): pass
    def on_batch_begin(self): pass
    def on_batch_end(self, raw_loss): pass

In [ ]:
class LossRecorder(Callback):
    beta = 0.98
    
    def on_train_begin(self):
        self.msa_loss,self.losses = 0,[]
        
    def on_batch_end(self, raw_loss):
        self.msa_loss = self.beta * self.msa_loss + (1-self.beta) * raw_loss
        self.losses.append(self.msa_loss / (1 - self.beta ** (len(self.losses)+1)))
        stop = False
        return stop

In [ ]:
class LRFinderCallback(LossRecorder):
    def __init__(self, opt, n_iter, start_lr, end_lr):
        self.sched = HyperParamScheduler(opt, 'lr', n_iter, annealing_exp, start_lr, end_lr)
        super().__init__()
        
    def on_train_begin(self):
        self.min_loss = 0
        super().on_train_begin()
    
    def on_batch_end(self, raw_loss):
        super().on_batch_end(raw_loss)
        loss = self.losses[-1]
        if len(self.losses) == 1 or loss < self.min_loss: self.min_loss = loss
        stop = self.sched.step()
        if loss > 4 * self.min_loss: return True
        return stop

In [ ]:
def loss_batch(model, xb, yb, loss_fn, opt=None):
    loss = loss_fn(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    return loss.item(), len(xb)

In [ ]:
def fit(epochs, model, loss_fn, opt, train_dl, valid_dl=None, callbacks=[]):
    for cb in callbacks: cb.on_train_begin()
    
    for epoch in range(epochs):
        model.train()
        for xb,yb in train_dl:
            for cb in callbacks: cb.on_batch_begin()
            raw_loss,_ = loss_batch(model, xb, yb, loss_fn, opt)
            if np.any([cb.on_batch_end(raw_loss) for cb in callbacks]): break  
                
        model.eval()
        if valid_dl is not None:
            with torch.no_grad():
                losses,nums = zip(*[loss_batch(model, xb, yb, loss_fn)
                                for xb,yb in valid_dl])
            val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)

            print(epoch, val_loss)
            
    for cb in callbacks: cb.on_train_end()

In [ ]:
class Learner():
    def __init__(self, data, model):
        self.data,self.model = data,model.to(data.device)
        self.loss_fn = F.nll_loss

    def fit(self, epochs, lr, opt_fn=optim.SGD):
        opt = opt_fn(self.model.parameters(), lr=lr)
        fit(epochs, self.model, self.loss_fn, opt, self.data.train_dl, self.data.valid_dl)
        
    def lr_find(self, opt_fn=optim.SGD, n_iter=200, start_lr=1e-5, end_lr=10):
        opt = opt_fn(self.model.parameters(), lr=start_lr)
        save_model(self.model, MODEL_PATH/'tmp.pt')
    
        cb = LRFinderCallback(opt, n_iter, start_lr, end_lr)
        epochs = int(np.ceil(n_iter/len(train_dl)))
        fit(epochs, self.model, self.loss_fn, opt, self.data.train_dl, callbacks=[cb])
    
        load_model(self.model, MODEL_PATH/'tmp.pt')
        return cb.sched.vals, cb.losses

In [ ]:
data = DataBunch.from_arrays(x_train,y_train,x_valid,y_valid, x_tfms=mnist2image)
model = Simple_CNN([1,16,16,10], [3,3,3], [2,2,2])
learner = Learner(data, model)

In [ ]:
lrs, losses = learner.lr_find()
plot_lr(lrs, losses)

## Training phases and SGDR

A few helper functions. The second one is there to map the names of the hyper-parameters in pytorch optimizers to 'momentum' and 'beta' (whether it's SGD, Adam, RMSProp)

In [ ]:
def is_tuple(x): return isinstance(x, tuple)

In [ ]:
def set_val_in_group(val, group, key):
    if key == 'momentum' and 'betas' in group:  group['betas'] = (val, group['betas'][1])
    elif key == 'beta' and 'betas' in group:    group['betas'] = (group['betas'][0], val)
    elif key == 'beta' and 'alpha' in group:    group['alpha'] = val
    else:                                       group[key] = val

In [ ]:
def set_val_in_groups(val, groups, key):
    for g in groups: set_val_in_group(val, g, key)

To simplify the use of training phase, the args lr and mom can either be:
- a single value, then it's assumed to have it constant during the phase
- a tuple (lr1, lr2), then the default is to go linearly from one to the other
- a tuple (lr1, lr2, VarType), then we go from one to the other using the VarType given.

In [ ]:
class TrainingPhase():
    
    def __init__(self, epochs, opt_fn, hp_dict):
        self.epochs,self.opt_fn,self.beta,self.wds = epochs,opt_fn,beta,wds
        self.start_lr, self.end_lr, self.lr_vType, self.extra_lr = self.decode(lr)
        self.start_mom, self.end_mom, self.mom_vType, self.extra_mom = self.decode(mom)
        
    def decode(self, vals):
        if is_tuple(vals):
            if len(vals) == 2: return vals + (VarType.LINEAR, None)
            elif len(vals) == 3: return vals + (None,)
            else: return vals
        else: return vals, None, VarType.NO, None
    
    def get_scheds(self, opt, n_batch):
        if self.beta is not None: set_val_in_groups(self.beta, opt.param_groups, 'beta')
        if self.wds is not None: set_val_in_groups(self.wds, opt.param_groups, 'weight_decay')
        return [Scheduler(opt, 'lr', n_batch * self.epochs, self.lr_vType, self.start_lr, self.end_lr, self.extra_lr),
               Scheduler(opt, 'momentum', n_batch * self.epochs, self.mom_vType, self.start_mom, self.end_mom, self.extra_mom)]

In [ ]:
LR = 'lr'
MOM = 'momentum'
WD = 'weight_decay'
BETA = 'beta'

In [ ]:
class TrainingPhase1():
    
    def __init__(self, epochs, opt_fn, **kwargs):
        self.epochs,self.opt_fn = epochs,opt_fn
        assert 'LR' in kwargs, 'You need to set a learning rate.'
        self.hp_behavior = {self.rename(n):self.decode(v) for n,v in kwargs.items()}
        self.start_lr = self.hp_behavior[LR][0]
        
    def decode(self, vals):
        if is_tuple(vals):
            if len(vals) == 2: return vals + (annealing_linear, None)
            elif len(vals) == 3: return vals + (None,)
            else: return vals
        else: return vals, None, annealing_no, None
        
    def rename(self, name):
        match = {'LR':LR, 'MOM':MOM, 'WD':WD, 'BETA':BETA}
        return match[name] if name in match else name
    
    def get_scheds(self, opt, n_batch):
        return [HyperParamScheduler(opt, n, int(n_batch * self.epochs), v[2], v[0], v[1], v[3]) for n,v in self.hp_behavior.items()]

In [ ]:
phase = TrainingPhase1(2, optim.SGD, LR=0.1, MOM=0.9, WD=1e-4)

The class that handles the phases and make sure we go from one to the next is another Callback.

In [ ]:
class PhaseScheduler(LossRecorder):
    
    def __init__(self, model, phases, n_batches):
        self.model, self.phases, self.n_batches = model, phases, n_batches
        super().__init__()
        self.phase = 0
        self.opt = None
        self.init_phase()
    
    def on_train_begin(self):
        super().on_train_begin()
        self.hp_vals = collections.defaultdict(list)
    
    def init_phase(self):
        cur_phase = self.phases[self.phase]
        #Updates the optimizer
        #TODO: Replacing self.model.parameters() by a function that returns the trainable parameters will handle freezing.
        new_opt = cur_phase.opt_fn(self.model.parameters(), lr=cur_phase.start_lr)
        if self.opt is None or type(self.opt) != type(new_opt): self.opt = new_opt
        #Creates the various schedulers
        self.scheds = cur_phase.get_scheds(self.opt, self.n_batches)
        
    def on_batch_end(self, raw_loss):
        super().on_batch_end(raw_loss)
        finished = False
        for sched in self.scheds: finished = finished or sched.step()
        if finished:
            for sched in self.scheds: self.hp_vals[sched.hyper_param] += sched.vals
            self.phase += 1
            if self.phase == len(self.phases): return True
            self.init_phase()

fitting with phases is then super easy

In [ ]:
def fit_phases(phases, model, loss_fn, train_dl, valid_dl=None, callbacks=[]):
    epochs = int(np.ceil(sum([p.epochs for p in phases])))
    cb = PhaseScheduler(model, phases, len(train_dl))
    fit(epochs, model, loss_fn, cb.opt, train_dl, valid_dl, callbacks=[cb])

An example of what a helper function could be.

In [ ]:
def sgdr_phases(lr, opt_fn, n_cycle, cyc_len, cyc_mul, mom=0.9, wds=None):
    phases = [TrainingPhase(0.05, opt_fn, lr/100, mom, wds)]
    for i in range(n_cycle):
        epochs = cyc_len - 0.05 if i==0 else cyc_len * (cyc_mul ** i)
        phases.append(TrainingPhase(epochs, opt_fn, (lr,0,VarType.COSINE), mom, wds))
    return phases

In [ ]:
def sgdr_phases1(lr, opt_fn, n_cycle, cyc_len, cyc_mul, mom=0.9, wds=None):
    phases = [TrainingPhase1(0.05, opt_fn, LR=lr/100, MOM=mom, WD=wds)]
    for i in range(n_cycle):
        epochs = cyc_len - 0.05 if i==0 else cyc_len * (cyc_mul ** i)
        phases.append(TrainingPhase1(epochs, opt_fn, LR=(lr,0,annealing_cos), MOM=mom, WD=wds))
    return phases

In [ ]:
class Learner():
    def __init__(self, data, model):
        self.data,self.model = data,model.to(data.device)
        self.loss_fn = F.nll_loss

    def fit(self, epochs, lr, opt_fn=optim.SGD):
        opt = opt_fn(self.model.parameters(), lr=lr)
        fit(epochs, self.model, self.loss_fn, opt, self.data.train_dl, self.data.valid_dl)
    
    def fit_phases(self, phases, model, loss_fn, train_dl, valid_dl=None, callbacks=[]):
        epochs = int(np.ceil(sum([p.epochs for p in phases])))
        cb = PhaseScheduler(model, phases, len(train_dl))
        fit(epochs, model, loss_fn, cb.opt, train_dl, valid_dl, callbacks=[cb])
        self.hp_vals = cb.hp_vals
        
    def lr_find(self, opt_fn=optim.SGD, n_iter=200, start_lr=1e-5, end_lr=10):
        opt = opt_fn(self.model.parameters(), lr=start_lr)
        save_model(self.model, MODEL_PATH/'tmp.pt')
    
        cb = LRFinderCallback(opt, n_iter, start_lr, end_lr)
        epochs = int(np.ceil(n_iter/len(train_dl)))
        fit(epochs, self.model, self.loss_fn, opt, self.data.train_dl, callbacks=[cb])
    
        load_model(self.model, MODEL_PATH/'tmp.pt')
        return cb.sched.vals, cb.losses

In [ ]:
data = DataBunch.from_arrays(x_train,y_train,x_valid,y_valid, x_tfms=mnist2image)
model = Simple_CNN([1,16,16,10], [3,3,3], [2,2,2])
learner = Learner(data, model)

In [ ]:
learner.fit_phases(sgdr_phases1(0.5, optim.SGD, 3, 1, 2, wds=1e-4), model, loss_fn, data.train_dl, data.valid_dl)

In [ ]:
def accuracy(model, val_dl):
    n, c = 0, 0
    for xb,yb in val_dl:
        probs = model(xb)
        preds = probs.argmax(1)
        n += (preds==yb).long().sum().item()
        c += yb.size(0)
    return n / c

In [ ]:
accuracy(learner.model, learner.data.valid_dl)

In [ ]:
plt.plot(learner.hp_vals[LR])

In [ ]:
plt.plot(learner.hp_vals[MOM])

In [ ]:
plt.plot(learner.hp_vals[WD])

# Sylvain's transforms

## Add transforms

In [ ]:
from enum import IntEnum

class TfmType(IntEnum):
    NO = 1
    PIXEL = 2
    COORD = 3
    CLASS = 4

In [ ]:
from abc import abstractmethod

class Transform():
    
    def __init__(self, tfm_y=TfmType.NO, p=1, batch_lvl = False):
        self.tfm_y,self.p,self.batch_lvl = tfm_y,p,batch_lvl
    
    def __call__(self, x, y):
        x,y = ((self.transform(x),y) if self.tfm_y==TfmType.NO
                else self.transform(x,y) if self.tfm_y in (TfmType.PIXEL, TfmType.CLASS)
                else self.transform_coord(x,y))
        return x, y
    
    def set_device(self, device):
        if not self.batch_lvl: self.device = device
    
    def transform_coord(self, x, y):
        if self.p == 1 or np.random.rand < self.p:
            return self.transform(x),y

    def transform(self, x, y=None):
        if self.p == 1 or np.random.rand < self.p:
            x = self.do_transform(x,False)
            return (x, self.do_transform(y,True)) if y is not None else x
        else: return x,y
    
    @abstractmethod
    def do_transform(self, x, is_y): raise NotImplementedError
    #In do_transform we can save a value (angle of a random rotation for instance) in self.save_for_y that will be used
    #if is_y is True.

In [ ]:
class ChannelOrder(Transform):
    #If we use PIL for data augmentation, maybe the conversion to a numpy array should be handled here?
    def __init__(self, tfm_y=TfmType.NO):
        super().__init__(tfm_y=tfm_y)
    
    def do_transform(self, x, is_y):
        if not is_y or self.tfm_y == TfmType.PIXEL: x = np.rollaxis(x, 2)
        return x

In [ ]:
class Normalize(Transform):
    
    def __init__(self, means, stds, tfm_y=TfmType.NO):
        self.means,self.stds = means,stds
        super().__init__(tfm_y=tfm_y, batch_lvl=True)
    
    def set_device(self, device):
        super().set_device(device)
        if type(self.means) != torch.Tensor or not self.means.device == device:
            self.means,self.stds = map(lambda x:torch.Tensor(x).to(device), (self.means, self.stds))
    
    def do_transform(self, x, is_y):
        if not is_y or self.tfm_y == TfmType.PIXEL:
            m, s = self.means[None,:,None,None].type_as(x), self.stds[None,:,None,None].type_as(x)
            x = (x - m) / s
        return x

In [ ]:
def compose(tfms, x, y):
    for tfm in tfms: x,y = tfm(x,y)
    return x,y

In [ ]:
def split_one_tfms(tfms):
    ds_tfms = [tfm for tfm in tfms if not tfm.batch_lvl]
    dl_tfms = [tfm for tfm in tfms if tfm.batch_lvl]
    return ds_tfms,dl_tfms
    
def split_tfms(trn_tfms, val_tfms):
    trn_ds_tfms, trn_dl_tfms = split_one_tfms(trn_tfms)
    val_ds_tfms, val_dl_tfms = split_one_tfms(val_tfms)
    return trn_ds_tfms, val_ds_tfms, trn_dl_tfms, val_dl_tfms